Run the cell below if you have never install it.

In [1]:
# !pip install openai==0.28
# !pip install openai

In [2]:
import pandas as pd
import os
import openai
import requests
import json

In [3]:
df1 = pd.read_csv("leeds_council_planning_apps_centroids_stx_y.csv")
df2 = pd.read_csv("leeds_hackathon_additional_data_fields.csv")

In [4]:
#df = df.drop('mnb_end_date', axis=1)

In [5]:
# df = df.dropna()
# rows_with_null = df[df.isnull().any(axis=1)]
# rows_with_null

In [10]:
df2.head()

,id,planning_id,reference,authority,proposal,heading,category,value_max,number_dwellings,stage,update_date,site_raw_address,link,geom_from,centroid_geom,easting,northing
0,4726464,15772391,23/07451/CLP,Leeds Council,Certificate of Proposed Lawful Development for...,Conversion to Dwelling,RESIDENTIAL,100.0,1.0,WITHDRAWN,2024-02-06 00:00:00.000,5 - 7 Recreation Place Holbeck Leeds LS11 0AN,https://publicaccess.leeds.gov.uk/online-appli...,postcode,POINT (429027.2335483082 431863.2733522415),429027.233548,431863.273352
1,4726464,15772391,23/07451/CLP,Leeds Council,Certificate of Proposed Lawful Development for...,Conversion to Dwelling,RESIDENTIAL,100.0,1.0,WITHDRAWN,2024-02-06 00:00:00.000,5 - 7 Recreation Place Holbeck Leeds LS11 0AN,https://publicaccess.leeds.gov.uk/online-appli...,postcode,POINT (429027.2335483082 431863.2733522415),429027.233548,431863.273352
2,4726461,15772299,23/07538/CLP,Leeds Council,Certificate of Proposed Lawful Development for...,Extension to Primary School,EDUCATION,100.0,NaN,DETAILED PLANS APPROVED,2024-02-06 00:00:00.000,Southroyd Primary School Littlemoor Crescent P...,https://publicaccess.leeds.gov.uk/online-appli...,postcode,POINT (422289.1438844915 432466.7482044691),422289.143884,432466.748204
3,4726461,15772299,23/07538/CLP,Leeds Council,Certificate of Proposed Lawful Development for...,Extension to Primary School,EDUCATION,100.0,NaN,DETAILED PLANS APPROVED,2024-02-06 00:00:00.000,Southroyd Primary School Littlemoor Crescent P...,https://publicaccess.leeds.gov.uk/online-appli...,postcode,POINT (422289.1438844915 432466.7482044691),422289.143884,432466.748204
4,4726462,15772324,23/07247/FU,Leeds Council,Demolition of existing garage; two storey side...,Two Storey Extension,EXTENSION/RENOVATION,100.0,NaN,DETAILED PLANS APPROVED,2024-02-06 00:00:00.000,1 The Beeches Pudsey LS28 7QS,https://publicaccess.leeds.gov.uk/online-appli...,postcode,POINT (420670.93662336015 433719.4311933177),420670.936623,433719.431193


In [6]:
openai.api_key = "set OPENAI_API_KEY=your-api-key-here"
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # ask a question here. it could be specific like the first example, or more general like the second example:
  ## example 1:
  #prompt = "What are the constraints of living within " + df + "meters of a flood zone?",
  
  ## example 2
  prompt = "Analyse this data: " + str(df2) +"And classify this 'proposal' column into different uses",
  
  temperature=0.7, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=1000, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question

print(response["choices"][0]["text"])
## supplementary datasets ##
# explore hmlr transactions: https://use-land-property-data.service.gov.uk/datasets/td



1. Conversion to Dwelling 
2. Extension to Primary School 
3. Two Storey Extension of Dwelling 
4. Extension of Dwelling 
5. Dwelling 
6. Extension to Dwelling 
7. New Build 
8. Demolition 
9. Renovation 
10. Conversion to Commercial Use 
11. Extension to Commercial Use 
12. Commercial Use 
13. Extension to Industrial Use 
14. Industrial Use 
15. Extension to Educational Use 
16. Educational Use 
17. Extension to Hospital 
18. Hospital 
19. Extension to Leisure Use 
20. Leisure Use 
21. Extension to Office Use 
22. Office Use 
23. Extension to Retail Use 
24. Retail Use 
 


In [7]:
openai.api_key = "set OPENAI_API_KEY=your-api-key-here"

# Extract the data from the 'heading' column as a list. This is just a demonstration; in actual use, data might need to be handled more appropriately.
headings = df2['heading'].tolist()
# To simplify the example, only the first few headings are selected for demonstration.
sample_headings = headings[:10]
formatted_headings = "\\n".join(sample_headings) # Format the heading list as a string

response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",  # Use the specified model
    prompt=f"Please classify the following headings into categories such as Residential, Commercial, Industrial, or Other:\\n{formatted_headings}",
    temperature=0.7,
    max_tokens=1000,
    top_p=1
)

print(response.choices[0].text)



Residential:
- Conversion to Dwelling
- Two Storey Extension
- One and Two Storey Extension
- One Storey Extension


#### Classify and summarize the proposals for planning applications

In [8]:
# Extract planning_id and proposal for the first 10 proposals as examples
sample_proposals = df2[['planning_id', 'proposal']].head(10)

# Convert the list of proposals into a formatted string understandable by the model, using planning_id for identification
formatted_proposals = "\n".join([f"{row['planning_id']}: {row['proposal']}" for index, row in sample_proposals.iterrows()])

# Construct the model prompt
prompt = f"Given the following list of planning proposals, classify each into categories such as Residential, Commercial, Industrial, or Other. Provide a brief explanation for each classification.\n\n{formatted_proposals}"

response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",  # Specify the model
    prompt=prompt,
    temperature=0,  # Adjust the temperature for more reliable answers
    max_tokens=500,  # Adjust the maximum number of tokens as needed
    top_p=1.0
)

# Print the model's classification suggestions
print(response.choices[0].text)



15772391: Residential - This proposal involves changing the use of a retail unit to a dwelling house, indicating that it will be used for residential purposes.

15772299: Other - This proposal involves the construction of a teaching block, which does not fit into the categories of residential, commercial, or industrial.

15772324: Residential - This proposal involves the demolition of a garage and the construction of a two storey side extension and front gable extension, indicating that it will be used for residential purposes.

15772361: Residential - This proposal involves the construction of a two storey and single storey side extension, indicating that it will be used for residential purposes.

15772442: Residential - This proposal involves the construction of a single storey rear extension, indicating that it will be used for residential purposes.


<br>
<br>

#### using `planning_id`, `proposal`, `category`, `heading`to summarize the planning apllication

In [11]:
# Extract 'planning_id', 'proposal', and 'category' for the first 10 proposals as examples
sample_data = df2[['planning_id', 'proposal', 'category', 'heading']].head(10)

# Format the list of proposals for the model, including 'planning_id' for identification
formatted_data = "\n".join([f"ID {row['planning_id']}: Proposal - {row['proposal']}, Category - {row['category']}, Heading - {row['heading']}" for index, row in sample_data.iterrows()])

# Construct the model prompt to include summarization of the 'proposal', classification into 'category', and consideration of the 'heading'
prompt = f"Given the following details of planning applications, summarize the intention and scope of each proposal based on its description, classify them into categories like Residential, Commercial, Industrial, or Other, and use the heading to provide context for each application.\n\n{formatted_data}"

# Call the API with the prompt to get summarizations and classifications
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature=0,  # Set to 0 for deterministic output
    max_tokens=300,  # Increased to allow space for summaries
    top_p=1.0
)

# Print the model's summarizations and category classifications
print(response.choices[0].text)



ID 15772391: Proposal - Certificate of Proposed Lawful Development for change of use from retail unit to dwelling house, Category - RESIDENTIAL, Heading - Conversion to Dwelling
Intention and Scope: The proposal is to change the use of a retail unit to a dwelling house. This would involve converting the existing space into a residential living space.
Category: Residential
Heading: Conversion to Dwelling

ID 15772299: Proposal - Certificate of Proposed Lawful Development for a replacement detached single storey teaching block to front with timber side terrace, Category - EDUCATION, Heading - Extension to Primary School
Intention and Scope: The proposal is to replace an existing teaching block with a new detached single storey building, including a timber side terrace. This would provide additional space for educational purposes at a primary school.
Category: Education
Heading: Extension to Primary School

ID 15772324: Proposal - Demolition of existing garage; two storey side extension

#### Classify the `proposal` column

In [12]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  prompt = "Analyse this data: " + str(df2) +"And classify this 'proposal' column into different uses",
  
  temperature=0, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=1000, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question

print(response["choices"][0]["text"])




The 'proposal' column can be classified into different uses based on the type of development proposed. Some possible categories could include:

1. Residential: This category would include proposals for new dwellings, conversions, extensions, and renovations of residential properties.

2. Commercial: This category would include proposals for new commercial buildings, such as offices, retail spaces, and industrial units.

3. Education: This category would include proposals for new schools, extensions to existing schools, and other educational facilities.

4. Leisure and Recreation: This category would include proposals for new leisure facilities, such as sports centers, gyms, and entertainment venues.

5. Infrastructure: This category would include proposals for new infrastructure projects, such as roads, bridges, and utilities.

6. Public Services: This category would include proposals for new public service buildings, such as hospitals, libraries, and government offices.

7. Mixed U

#### Classify the `heading` column

In [13]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  prompt = "Analyse this data: " + str(df2) +"And classify this 'heading' column into different uses",
  
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=1000, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question

print(response["choices"][0]["text"])



The 'heading' column can be classified into the following categories based on the information provided:

1. Conversion/renovation: This category includes headings such as 'Conversion to Dwelling' and 'Two Storey Extension', which indicate that the proposed development involves converting an existing structure or making changes to an existing building.

2. New build: This category includes headings such as 'Dwelling' and 'New Build', which indicate that the proposed development involves the construction of a new building.

3. Extension: This category includes headings such as 'One and Two Storey Extension' and 'Loft Conversion', which indicate that the proposed development involves extending an existing building.

4. Education: This category includes headings such as 'Extension to Primary School' and 'School Development', which indicate that the proposed development involves building or expanding educational facilities.

5. Residential: This category includes headings such as 'Convers

#### Summarize the overall trend of planning applications

In [14]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this

  prompt = "Analyse this data: " + str(df2) + "Summarize the overall trend of planning applications",
  
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=1000, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question

print(response["choices"][0]["text"])

 in Leeds

The overall trend of planning applications in Leeds is that there has been a steady increase in the number of applications over the years. From 2024-02-06, there were 28871 planning applications submitted to Leeds Council. This is a significant increase from previous years, indicating a growing demand for development and construction in the city. The majority of applications are for residential projects, followed by education and extension/renovation projects. The most common stage for applications is "detailed plans approved," indicating that most projects have been approved by the council. There is also a mix of new build and renovation projects, with a maximum value of 200 for new build projects and 100 for renovation projects. The data also shows that there is a mix of single and multiple dwelling projects, with the majority being single dwellings. Overall, the trend suggests a growing and diverse demand for development in Leeds.


In [16]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this

  prompt = "Analyse this data: " + str(df2) + "Identify all residential planning applications and summarize their key features.",
  
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=500, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question

print(response["choices"][0]["text"])



The data provided contains information on 28,871 planning applications submitted to Leeds Council. Of these, 16,255 (56.3%) are classified as residential, with a majority (10,912 or 67.2%) being for extensions or renovations.

The average maximum value of these residential applications is 100, with a range of 100 to 200. The average number of dwellings proposed in these applications is 1, with a range of 1 to 6.

The most common authority for these applications is Leeds Council, accounting for all 16,255 residential applications. The most common stage for these applications is "detailed plans approved", accounting for 11,228 (69.1%) of the residential applications.

The most common category for these applications is "extension/renovation", accounting for 10,912 (67.2%) of the residential applications. The most common proposal for these applications is "certificate of proposed lawful development", accounting for 7,226 (44.4%) of the residential applications.

The most common heading f

<br>
<br>
<br>

## Planning Applications within a 250m Radius of Location 1

In [3]:
df3 = pd.read_csv('loc1_250m2.csv')

In [6]:
# Select specific columns
df3_selected = df3.loc[:, ['id', 'planning_id', 'authority', 'proposal', 'heading', 'category', 'value_max', 
                           'stage', 'update_date', 'site_raw_address', 'imported_at', 'distance']]

df3_selected.head()

,id,planning_id,authority,proposal,heading,category,value_max,stage,update_date,site_raw_address,imported_at,distance
0,4714806,15734308,Leeds Council,Change of use from offices (Class E(g) to acco...,Change of use from offices (Class E(g...,OTHER,100.0,DETAILED PLANS APPROVED,2024-01-31 00:00:00.000,1 Wellington Place Leeds LS1 4AP,2024-02-02 04:09:21.220 +0000,194.626639
1,4714806,15734308,Leeds Council,Change of use from offices (Class E(g) to acco...,Conversion to Accountancy Training Facuility,EDUCATION,100.0,DETAILED PLANS APPROVED,2024-01-31 00:00:00.000,1 Wellington Place Leeds LS1 4AP,2024-02-02 04:09:21.220 +0000,194.626639
2,4764288,15825085,Leeds Council,Enabling works associated with office developm...,Office,COMMERCIAL,100.0,DETAILED PLANS SUBMITTED,2024-03-06 00:00:00.000,9 Wellington Place Leeds LS1 4AP,2024-02-29 03:47:06.994 +0000,194.626639
3,4183586,13348868,Leeds Council,Change of use of office to six apartments,Conversion to 6 Apartments,RESIDENTIAL,400.0,WITHDRAWN,2018-06-27 23:00:00.000,Part Ground Floor Riverside West Whitehall Roa...,2023-04-04 01:43:51.735 +0100,7.634203
4,2756829,14063967,Leeds Council,Application for enabling works to facilitate t...,Demolition,OTHER,100.0,DETAILED PLANS SUBMITTED,2019-12-16 00:00:00.000,Land At Whitehall Road And Globe Road Leeds LS...,2023-03-23 14:13:20.390 +0000,229.220011


In [7]:
# Convert DataFrame to string
df3_str = df3_selected.to_string(index=False)

#### 1. Understanding the Overall Trend of Planning Applications  
⟶ “Summarize the overall trend of planning applications”

In [19]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Summarize the overall trend of planning applications based on the following data:\n" + df3_str + "\n\n",
    
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=200, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])

The overall trend of planning applications is that there has been an increase in applications for development projects in Leeds. This can be seen in the increasing number of applications over the years, with a peak in 2023. The majority of these applications are for mixed-use developments, commercial developments, and residential conversions. There is also a trend towards larger developments, with higher values and more detailed plans being submitted. The stage of the applications also indicates that many of these developments have been approved and are in the detailed planning stage. This trend suggests that there is a growing interest in development and investment in Leeds, and the city is experiencing significant growth and development.


In [9]:
openai.api_key = "set OPENAI_API_KEY=your-api-key-here"
 
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """
            You are now a Friendly Real Estate Guide for homebuyers, tasked with explaining the latest trends in Leeds' housing market in the simplest terms.
            Imagine you're talking to someone with no real estate background. 
            Your goal is to make them feel informed and comfortable with the current housing landscape, focusing on:
            
            - What types of homes are being built?
            - How are these homes helping the environment?
            - Where in Leeds are these developments happening?
            - Are a lot of these projects getting approved?
            - What’s being done to make living in Leeds better, like parks or internet upgrades?
            Please use simple language and examples to bring these points to life. Think of how you would explain it to a friend curious about moving to Leeds, highlighting what's exciting and important for a future homebuyer.
            """
        },
        {
            "role": "user",
            "content": df3_str
        }
    ],
    temperature=0,
    max_tokens=1000,
    top_p=1
)
 
print(response["choices"][0]["message"]["content"])

Hey there! Looking to move to Leeds? Let me give you the lowdown on what's happening in the housing market there.

**Types of Homes Being Built:**
- In Leeds, there's a mix of developments going on. You've got residential apartments, mixed-use buildings with a combination of homes and commercial spaces, and even purpose-built student accommodations. For example, there are plans for buildings with hundreds of flats or apartments, ranging from 245 units to 783 dwellings.

**Environmentally Friendly Homes:**
- Many new developments are focusing on being eco-friendly. Some projects include landscaping, public realm works, and even flood defense constructions to protect the environment. There are also installations like external louvres, replacement of glazed panels with more energy-efficient options, and even 5G telecoms installations to improve connectivity sustainably.

**Locations of Developments:**
- These exciting projects are spread across Leeds, from Wellington Place to Whitehall Ro

In [11]:
openai.api_key = "set OPENAI_API_KEY=your-api-key-here"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """
            You are now a Friendly Real Estate Analyst, specialized in deciphering complex data for homebuyers.
            Based on the dataset:
            """
            + df3_str +
            """"
            containing planning applications within a 250-meter radius of a specific property in Leeds.
            The data includes application types, submission dates, approval statuses, and more.
            Your task is to analyze this dataset to uncover the overall trends in this area 
            that would be significant to a potential homebuyer. Focus on the types of developments being proposed,
            the rate at which applications are approved, any emphasis on sustainability features, and the distribution 
            of these projects within the specified radius. Simplify your analysis into key points that would be exciting
            and important for someone considering a move to this part of Leeds.
            """
        },
        {
            "role": "user",
            "content": "Please analyze" + df3_str + "to identify the overall trends in planning applications around the selected Leeds property."
        }
    ],
    temperature=0,
    max_tokens=1000,
    top_p=1
)

print(response["choices"][0]["message"]["content"])

Based on the provided dataset of planning applications around the selected property in Leeds, here are the key trends and insights:

1. **Types of Developments**:
   - The proposed developments include a variety of project types such as office conversions, residential developments, mixed-use buildings, educational facilities, infrastructure installations, landscaping projects, and demolition works.
   - There is a mix of commercial, residential, educational, and infrastructure-related developments in the area.

2. **Approval Status**:
   - The majority of the planning applications have detailed plans approved, indicating a positive trend in the approval of proposed developments.
   - Some applications have been withdrawn, which suggests that not all proposed projects move forward to completion.

3. **Sustainability Features**:
   - There is a focus on sustainability features such as recladding of existing buildings, installation of external louvres for energy efficiency, creation of ri

In [15]:
openai.api_key = "set OPENAI_API_KEY=your-api-key-here"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        
        {
            "role": "system",
            "content": """
            As a Friendly Real Estate Analyst, simplify complex planning application data within a 250-meter radius of a specified property in Leeds for potential homebuyers. Use only the provided text, avoiding any external information. Your goal is to distill the essence of the data, focusing sharply on:
            - The types of developments (based on the category column).
            - Any clear trends towards the type of use.
            - Notable features related to sustainability or infrastructure improvements.
            Produce a digest that is not only brief and straightforward but also highlights the main points in an easily digestible format for non-experts. Aim for bullet points or a short, clear paragraph that captures the essence without technical jargon.
        """
        + "Based on this dataset: " + df3_str +
        """
        Specifically, extract and highlight the most impactful insights relevant to homebuyers in this area, making it as precise and reader-friendly as possible.
        """
        },
        {
            "role": "user",
            "content": "I need a succinct overview of the key trends in planning applications around the chosen Leeds property, focusing only on the provided dataset. What should I know?"
        }
    ],
    temperature=0,
    max_tokens=500,  # Reduce max tokens if too lengthy
    top_p=1
)
print(response["choices"][0]["message"]["content"])

- **Development Types:**
  - Majority of developments are related to **commercial** properties such as offices, mixed-use developments, and conversions to office spaces.
  - Residential developments include conversions to apartments and the construction of residential blocks.

- **Trends:**
  - A noticeable trend towards **commercial** use developments.
  - Significant focus on **office** spaces and **mixed-use** developments.
  - Some applications involve **infrastructure improvements** like flood defense, car parks, and landscaping.

- **Notable Features:**
  - Emphasis on **sustainability** with applications for **demolition** and **recladding** for energy efficiency.
  - Infrastructure enhancements include **telecoms installations**, **flood defense**, and **riverside walks**.
  - Diverse range of developments catering to different needs from **education** to **leisure** establishments.


In [21]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """
            As a Friendly Real Estate Analyst, your task is to succinctly summarize the planning applications
            within a 250-meter radius of a specified property in Leeds for potential homebuyers.
            Focus strictly on the provided dataset without external information. Highlight the most critical insights
            in the briefest manner possible, such as:
            - Key types of developments.
            - Any apparent trends in usage.
            - Notable sustainability or infrastructure features.
            Keep your summary extremely concise, aiming for bullet points or a very short paragraph.
            Emphasize clarity and brevity, avoiding any technical jargon.
            Based on this dataset: """ + df3_str + """
            Your goal is to distill the essence of the trends in a way that's most accessible to someone interested in
            purchasing a home in this area.
            Please provide the insights in no more than three bullet points or a single, short paragraph.
            """
        },
        {
            "role": "user",
            "content": """
            I'm looking for a sharp, concise overview of the main trends in planning applications around the 
            selected Leeds property. What are the essentials?"""
        }
    ],
    temperature=0,
    max_tokens=500,
    top_p=1
)

print(response["choices"][0]["message"]["content"])

- A significant trend in the planning applications around the Leeds property is the conversion of office spaces into different uses such as accountancy training facilities, apartments, and drinking establishments, reflecting a shift in the area's utilization.
- There is a notable focus on mixed-use developments combining residential units with commercial spaces, indicating a desire for diverse and integrated urban environments.
- Infrastructure enhancements like new car parks, external louvres, and landscaping projects are prevalent, suggesting a commitment to improving the area's amenities and aesthetics.


#### 2. Identify all residential planning applications and summarize their key features  
⟶ “Identify all residential planning applications and summarize their key features”

In [20]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """
            As a Friendly Real Estate Analyst, your task is to succinctly summarize the planning applications
            within a 250-meter radius of a specified property in Leeds for potential homebuyers.
            Focus strictly on the provided dataset without external information. Highlight the most critical insights
            in the briefest manner possible, such as:
            - Key types of developments.
            - Any apparent trends in usage.
            - Notable sustainability or infrastructure features.
            Keep your summary extremely concise, aiming for bullet points or a very short paragraph.
            Emphasize clarity and brevity, avoiding any technical jargon.
            Based on this dataset: """ + df3_str + """
            Your goal is to distill the essence of the trends in a way that's most accessible to someone interested in
            purchasing a home in this area.
            Please provide the insights in no more than three bullet points or a single, short paragraph.
            """
        },
        {
            "role": "user",
            "content": """
            I'm looking for a sharp, concise overview of the main trends in planning applications around the 
            selected Leeds property. What are the essentials?"""
        }
    ],
    temperature=0,
    max_tokens=500,
    top_p=1
)

print(response["choices"][0]["message"]["content"])


1. ID: 4764288
   Planning ID: 15825085
   Reference: 24/00747/FU
   Authority: Leeds Council
   Proposal: Enabling works associated with office development proposed under 22/00775/RM
   Heading: Office
   Category: Commercial
   Value Maximum: 100
   Stage: Detailed Plans Submitted
   Update Date: 00:00.0
   Site Raw Address: 9 Wellington Place Leeds LS1 4AP

2. ID: 4183586
   Planning ID: 13348868
   Reference: 18/02538/DPD
   Authority: Leeds Council
   Proposal: Change of use of office to six apartments
   Heading: Conversion to 6 Apartments
   Category: Residential
   Value Maximum: 400
   Stage: Withdrawn
   Update Date: 00:00.0
   Site Raw Address: Part Ground


#### 3. Analysis of the Impact of Planning Applications  
⟶ “Analyse the potential impact of recent commercial planning applications on the residential community”

In [22]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Make it concise and clear: Analyze the potential impact of recent commercial planning applications on the residential community and provide concise guidance for homebuyers based on the following data:\n" + df3_str + "\n\n",
    
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=300, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])


Recent commercial planning applications may have a significant impact on the residential community. These applications include proposals for office developments, conversion of offices to apartments, mixed-use developments, and installation of infrastructure such as telecoms and flood defense. Homebuyers should carefully consider the potential impact of these developments on their quality of life, access to amenities, and property values. It is recommended that homebuyers research the details of these planning applications, including the type of development, proposed number of units, and any associated infrastructure or landscaping plans. They should also consider the stage of the application process and any updates or changes that may have occurred. It is important for homebuyers to thoroughly evaluate the potential impact of these commercial planning applications before making a decision on purchasing a property in the affected area.


#### 4. Status of Planning Application Approvals  
⟶ “List the planning applications that are currently pending approval and describe their potential benefits and drawbacks.”


In [ ]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Make it concise and clear: List the planning applications that are currently pending approval and describe their potential benefits and drawbacks based on the following data:\n" + df3_str + "\n\n",
    
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=300, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])

#### 5. Details of Recently Approved Planning Applications  
⟶ “Provide details of the most recently approved planning applications and their implications for future development”

In [ ]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Summarize the impact of newly approved planning applications for homebuyers in Leeds, focusing on property values and living quality. See data:\n" + df3_str + "\n\n",
    
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=350, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print("Summary of recent approved planning applications in Leeds and their potential implications for future development:")
print(response["choices"][0]["text"])

#### 6. Environmental Considerations in Planning Applications  
⟶ “Identify planning applications with key environmental sustainability features that benefit homebuyers, such as energy efficiency, use of renewable resources, and sustainable materials.”

In [ ]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Identify planning applications with key environmental sustainability features that benefit homebuyers, such as energy efficiency, use of renewable resources, and sustainable materials, from the following data:\n" + df3_str + "\n\n",
    
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=300, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])

#### 7. Planning Applications and Municipal Facilities  
⟶ “Analyze how specific planning applications might enhance the living environment for residents, focusing on infrastructure improvements and the addition of public amenities. Highlight the planning applications that could significantly impact property values and the attractiveness of neighborhoods for homebuyers”

In [ ]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Highlight planning applications with positive impacts on infrastructure and amenities for homebuyers. Data:\n" + df3_str + "\n\n",
  
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=350, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])

#### 8. Comparing the Economic Values of Different Planning Applications  
⟶ “Compare the economic values of planning applications and identify the highest-value projects.”

In [ ]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Identify the highest-value residential projects from the planning applications that are most beneficial for homebuyers, and list them in descending order of economic value based on the provided data. Data:\n" + df3_str + "\n\n",
  
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=350, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])

Other prompt I tried:

In [ ]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "Is it worth buying a house in this area? Data:\n" + df3_str + "\n\n",
  
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=350, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])

In [ ]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct", # do not change this
  
  # Create prompt
  prompt = "What is the average house price in this area? Data:\n" + df3_str + "\n\n",
  
  temperature=0.5, # temperature ranges from 0-2, with 2 being more random. do NOT change both temperature and top_p.
  max_tokens=350, #2048 is maximum allowed
  top_p=1 #model considers the results of the tokens with top_p probability mass; 0.1 means only the tokens comprising the top 10% probability mass are considered
)

# print the answer to the question
print(response["choices"][0]["text"])